In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [5]:
dexcom = pd.read_csv("/content/drive/My Drive/REU 2024/datafiles/002/Dexcom_002.csv")
HR = pd.read_csv("/content/drive/My Drive/REU 2024/datafiles/002/HR_002.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/REU 2024/datafiles/002/Dexcom_002.csv'

In [ ]:
glucose_start = "2020-02-21 11:08:36"
glucose_end = dexcom["Timestamp (YYYY-MM-DDThh:mm:ss)"]
glucose_end = glucose_end.iloc[-1]


glucose_starting_index = list(dexcom["Timestamp (YYYY-MM-DDThh:mm:ss)"]).index(glucose_start)
glucose_ending_index = list(dexcom["Timestamp (YYYY-MM-DDThh:mm:ss)"]).index(glucose_end)
glucose = dexcom.loc[glucose_starting_index : glucose_ending_index + 1]
glucose = glucose[["Timestamp (YYYY-MM-DDThh:mm:ss)", glucose.columns[7]]]
glucose['Timestamp (YYYY-MM-DDThh:mm:ss)'] = pd.to_datetime(glucose['Timestamp (YYYY-MM-DDThh:mm:ss)'])
glucose.set_index('Timestamp (YYYY-MM-DDThh:mm:ss)', inplace=True)

,Glucose Value (mg/dL)
Timestamp (YYYY-MM-DDThh:mm:ss),
2020-02-21 11:05:00,186.0
2020-02-21 11:10:00,181.0
2020-02-21 11:15:00,175.0
2020-02-21 11:20:00,168.0
2020-02-21 11:25:00,163.0
...,...
2020-02-29 09:15:00,177.0
2020-02-29 09:20:00,171.0
2020-02-29 09:25:00,166.0


In [ ]:
HR_starting_index = list(HR[HR.columns[0]]).index(glucose_start)
HR_ending_index = list(HR[HR.columns[0]]).index(glucose_end)
heart_rates = HR.loc[HR_starting_index:HR_ending_index]
heart_rates['datetime'] = pd.to_datetime(heart_rates['datetime'])
heart_rates.set_index("datetime", inplace=True)


C:\Users\duyth\AppData\Local\Temp\ipykernel_26384\1520964633.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heart_rates['datetime'] = pd.to_datetime(heart_rates['datetime'])


,hr
datetime,
2020-02-21 11:05:00,89.571786
2020-02-21 11:10:00,88.518300
2020-02-21 11:15:00,87.909533
2020-02-21 11:20:00,88.003800
2020-02-21 11:25:00,86.766700
...,...
2020-02-29 09:15:00,85.025900
2020-02-29 09:20:00,83.188767
2020-02-29 09:25:00,84.574767


In [ ]:
interpolated_glucose = glucose.resample('1S').interpolate()
interpolated_glucose['datetime'] = interpolated_glucose.index
interpolated_glucose.reset_index(drop=True, inplace=True)
interpolated_hr = heart_rates.resample('1S').interpolate()
interpolated_hr['datetime'] = interpolated_hr.index
interpolated_hr.reset_index(drop=True, inplace=True)

interpolated_hr = interpolated_hr[['datetime', ' hr']]
interpolated_glucose = interpolated_glucose[['datetime', 'Glucose Value (mg/dL)']]

Accelerometry Processing

In [ ]:
acc = pd.read_csv('.//datafiles//002//ACC_002.csv')

In [ ]:

def magnitude(x, y, z):
  return (x**2 + y**2 + z**2)**(1/2)

acc['datetime'] = pd.to_datetime(acc['datetime'])
acc.set_index('datetime', inplace=True)

ds_acc = acc.resample('1S').mean()
glucose_start = pd.to_datetime(glucose_start)
glucose_end = pd.to_datetime(glucose_end)
ds_acc = ds_acc.loc[glucose_start:glucose_end]
ds_acc['datetime'] = ds_acc.index
ds_acc.reset_index(drop = True, inplace = True)
ds_acc['magnitude'] = ds_acc.apply(lambda row: magnitude(row[' acc_x'], row[' acc_y'], row[' acc_z']), axis=1)

In [ ]:
ds_acc

,acc_x,acc_y,acc_z,datetime,magnitude
0,-23.00000,10.0,58.12500,2020-02-21 11:08:36,63.304942
1,-23.00000,10.0,58.06250,2020-02-21 11:08:37,63.247560
2,-23.00000,10.0,58.09375,2020-02-21 11:08:38,63.276250
3,-22.87500,10.0,58.25000,2020-02-21 11:08:39,63.374507
4,-22.71875,10.0,58.37500,2020-02-21 11:08:40,63.433290
...,...,...,...,...,...
685784,-64.00000,6.0,13.28125,2020-02-29 09:38:20,65.638339
685785,-64.00000,6.0,13.12500,2020-02-29 09:38:21,65.606902
685786,-64.00000,6.0,13.00000,2020-02-29 09:38:22,65.582010
685787,-64.00000,6.0,13.00000,2020-02-29 09:38:23,65.582010


In [ ]:
bvp = pd.read_csv('.//datafiles//002//BVP_002.csv')

In [ ]:
bvp['datetime'] = pd.to_datetime(bvp['datetime'])
bvp.set_index('datetime', inplace=True)
ds_bvp = bvp.resample('1S').mean()
del bvp
ds_bvp['datetime'] = ds_bvp.index
ds_bvp.reset_index(drop=True,inplace=True)

bvp_start_index = list(ds_bvp['datetime']).index(glucose_start)
bvp_end_index = list(ds_bvp['datetime']).index(glucose_end)

ds_bvp = ds_bvp.loc[bvp_start_index : bvp_end_index]
ds_bvp


,bvp,datetime
6570,2.884844,2020-02-21 11:08:36
6571,-2.046250,2020-02-21 11:08:37
6572,-0.407344,2020-02-21 11:08:38
6573,-1.387812,2020-02-21 11:08:39
6574,5.483281,2020-02-21 11:08:40
...,...,...
692354,2.546250,2020-02-29 09:38:20
692355,-3.859375,2020-02-29 09:38:21
692356,0.997500,2020-02-29 09:38:22
692357,1.700312,2020-02-29 09:38:23


In [ ]:
eda = pd.read_csv('.//datafiles//002//EDA_002.csv')

In [ ]:
eda['datetime'] = pd.to_datetime(eda['datetime'])
eda.set_index('datetime', inplace=True)
ds_eda = eda.resample('1S').mean()

del eda
ds_eda['datetime'] = ds_eda.index
ds_eda.reset_index(drop=True, inplace=True)

eda_start_index = list(ds_eda['datetime']).index(glucose_start)
eda_end_index = list(ds_eda['datetime']).index(glucose_end)

ds_eda = ds_eda.loc[eda_start_index : eda_end_index]
ds_eda

,eda,datetime
6570,0.303906,2020-02-21 11:08:36
6571,0.302625,2020-02-21 11:08:37
6572,0.301984,2020-02-21 11:08:38
6573,0.301984,2020-02-21 11:08:39
6574,0.301343,2020-02-21 11:08:40
...,...,...
692354,0.480428,2020-02-29 09:38:20
692355,0.481389,2020-02-29 09:38:21
692356,0.479787,2020-02-29 09:38:22
692357,0.478826,2020-02-29 09:38:23


In [ ]:
ibi = pd.read_csv('.//datafiles//002//IBI_002.csv')
ibi['datetime'] = pd.to_datetime(ibi['datetime'])
ibi.set_index('datetime', inplace=True)
ds_ibi = ibi.resample('1S').mean()

ds_ibi['datetime'] = ds_ibi.index
ds_ibi.reset_index(drop=True, inplace=True)

ibi_start_index = list(ds_ibi['datetime']).index(glucose_start)
ibi_end_index = list(ds_ibi['datetime']).index(glucose_end)

ds_ibi = ds_ibi.loc[ibi_start_index : ibi_end_index]
ds_ibi

,ibi,datetime
6542,0.687531,2020-02-21 11:08:36
6543,0.679719,2020-02-21 11:08:37
6544,0.671906,2020-02-21 11:08:38
6545,0.648467,2020-02-21 11:08:39
6546,0.671906,2020-02-21 11:08:40
...,...,...
692326,0.710970,2020-02-29 09:38:20
692327,0.734409,2020-02-29 09:38:21
692328,0.734409,2020-02-29 09:38:22
692329,0.757847,2020-02-29 09:38:23


In [ ]:
temp = pd.read_csv('.//datafiles//002//TEMP_002.csv')
temp['datetime'] = pd.to_datetime(temp['datetime'])
temp.set_index('datetime', inplace=True)
ds_temp = temp.resample('1S').mean()

ds_temp['datetime'] = ds_temp.index
ds_temp.reset_index(drop=True, inplace=True)

glucose_start = pd.to_datetime(glucose_start)
glucose_end = pd.to_datetime(glucose_end)
temp_start_index = list(ds_temp['datetime']).index(glucose_start)
temp_end_index = list(ds_temp['datetime']).index(glucose_end)

ds_temp = ds_temp.loc[temp_start_index : temp_end_index]
ds_temp

,temp,datetime
6570,34.13,2020-02-21 11:08:36
6571,34.13,2020-02-21 11:08:37
6572,34.11,2020-02-21 11:08:38
6573,34.13,2020-02-21 11:08:39
6574,34.11,2020-02-21 11:08:40
...,...,...
692354,32.99,2020-02-29 09:38:20
692355,33.00,2020-02-29 09:38:21
692356,32.99,2020-02-29 09:38:22
692357,32.99,2020-02-29 09:38:23
